# Chapter 12: Inheritance: For Good or For Worse

This chapter is about inheritance and subclassing, discussing good and bad practices when building class hierarchies. Chapter 12 emphasises two particulars very specific to Python:

* The pitfalls of subclassing from built-in types.

* Multiple inheritance and the method resolution order.

Multiple inheritance will be illustrated using two important Python projects: the TKinter GUI toolkit and the Django Web framework.

## Subclassing Built-In Types is Tricky

Since Python 2.2 it has been possible to subclass built-in types such as `list` or `dict`. The is **one major caveat**: the code of the built-ins (written in C) does not all special methods overriden by user-defined classes.

In [4]:
# Example 12-1. __setitem__ override is ignored by __init__ and __update__

class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)
        
dd = DoppelDict(one=1)
dd

{'one': 1}

In [5]:
dd["two"] = 2
dd

{'one': 1, 'two': [2, 2]}

In [6]:
dd.update(three=3)
dd

{'one': 1, 'two': [2, 2], 'three': 3}

#### Things to note about the above implementation:

* `[]` calls our `__setitem__` and works as expected.

* The `update` method from `dict` does not use our version of `__setitem__`.

* This built-in behaviour violates a basic rule of OOP: the search for methods should always start from the class of the target instance (`self`), even when the call happens inside a method implemented in a superclass.

**Takeaway:** Subclassing built-ins is error-prone. Instead, derive classes from `collections` module using `UserDict`, `UserList`, and `UserString` which are designed to be easily extended.

In [7]:
# Example 12-3. DoppelDict now works as expected because it extends UserDict and not dict

import collections

class DoppelDict(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)
        
dd = DoppelDict(one=1)
dd

{'one': [1, 1]}

In [8]:
dd["two"] = 2
dd

{'one': [1, 1], 'two': [2, 2]}

In [9]:
dd.update(three=3)
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

## Multiple Inheritance and Method Resolution Order

TODO.